In [ ]:
from random import choices, randint, randrange, random, sample
from typing import List, Optional, Callable, Tuple, Union

Genome = List[Union[List[int], float]]
Population = List[Genome]
FitnessFunc = Callable[[Genome], int]
PopulateFunc = Callable[[int], Population]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]
PrinterFunc = Callable[[Population, int, FitnessFunc], None]

number_map = {0: "C", 1: "C#", 2: "Db", 2: "D", 3: "Eb", 4: "E", 5: "F", 6: "Gb", 7: "G", 8: "Ab", 9: "A", 10: "A#", 11: "B"}
note_map = {"C": 0, "C#": 1, "Db": 1, "D": 2, "Eb": 3, "E": 4, "F": 5, "F#": 6, "Gb": 6, "G": 7, "G#": 8, "Ab": 8, "A": 9, "A#": 10, "Bb": 10, "B": 11}

def generate_genome(len_of_tune) -> Genome:
    note_list = []  # stores notes as LETTERS
    length_list = []  # stores lengths of each note as FLOATS
    possible_lengths = [0.25, 0.5, 0.75]  # lengths are 0.25, 0.5, and 0.75

    # TODO: populate the note_list and length_list randomly.
    # hint: use the note_map dictionary
    for n in range(len_of_tune):
        note_list.append(note_map[number_map[randint(0, 11)]])
    length_list.append(choices(possible_lengths, k=len_of_tune))

    # TODO 2: incorporate scales to enhance your random melodies.
    # hint: use the find_scales method from the previous block of code
    # Note: start by incorporating a singular scale.
    tune_length = len_of_tune
    random_scale = find_scale(number_map[randint(0, 11)])
    while tune_length > 0:
        random_note = number_map[randint(0, 11)]
        if random_note in random_scale:
            note_list.append(note_map[random_note])
            tune_length -= 1

    ### DO NOT CHANGE ###
    genome = []
    for i in range(0, 2 * len_of_tune):
        if i % 2 == 0:
            genome.append(note_list[i // 2])
        else:
            genome.append(length_list[(i - 1) // 2])
    return genome
    ### DO NOT CHANGE ###

def generate_population(size: int, genome_length: int) -> Population:
    population = []
    for i in range(size):
        population.append(generate_genome(genome_length))
    return population

def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")

    length = len(a)
    if length < 2:
        return a, b

    p = randint(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]

def letter_mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    for i in range(num):
        if random() > probability:  # then we perform the mutation
            index = randint(0, len(genome) - 1)
            if type(genome[index]) is float:
                genome[index] = 0.25 * randint(1, 4)
            else:
                genome[index] = number_map[randint(0, 11)]
    return genome

def population_fitness(population: Population, fitness_func: FitnessFunc) -> int:
    return sum(fitness_func(genome) for genome in population)

def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    return sample(
        population=generate_weighted_distribution(population, fitness_func),
        k=2
    )

def generate_weighted_distribution(population: Population, fitness_func: FitnessFunc) -> Population:
    result = []
    for gene in population:
        result += [gene] * int(fitness_func(gene) + 1)
    return result

def sort_population(population: Population, fitness_func: FitnessFunc) -> Population:
    return sorted(population, key=fitness_func, reverse=True)

def genome_to_string(genome: Genome) -> str:
    return "".join(map(str, genome))

def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc):
    print("GENERATION %02d" % generation_id)
    print("---------------")
    print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
    print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print("Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]), fitness_func(sorted_population[-1])))
    print("")
    return sorted_population[0]

def run_evolution(
    populate_func: PopulateFunc,
    fitness_func: FitnessFunc,
    selection_func: SelectionFunc = selection_pair,
    crossover_func: CrossoverFunc = single_point_crossover,
    mutation_func: MutationFunc = letter_mutation,
    generation_limit: int = 100,
    fitness_limit: int = None,
    printer: Optional[PrinterFunc] = None
) -> Tuple[Population, int]:
    population = populate_func()
    i = 0
    for i in range(generation_limit):
        population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)

        if printer is not None:
            printer(population, i, fitness_func)

        if fitness_func(population[0]) >= fitness_limit:
            print("reached fitness limit")
            break

        next_generation = population[0:2]  # transfer the two fittest beings
        for j in range(int((len(population) / 2) - 1)):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation
        print("NEXT GEN")

    return population, i

# Example usage
if __name__ == "__main__":
    # Define example functions here to test the genetic algorithm
    def find_scale(note: int) -> List[int]:
        """Example function to find a scale containing the given note."""
        return [note, (note + 2) % 12, (note + 4) % 12, (note + 5) % 12, (note + 7) % 12, (note + 9) % 12, (note + 11) % 12]

    def fitness_func(genome: Genome) -> int:
        """Example fitness function."""
        return sum(1 for gene in genome if type(gene) is int and gene in [0, 2, 4, 5, 7, 9, 11])  # counts C, D, E, F, G, A, B

    population, generations = run_evolution(
        populate_func=lambda: generate_population(10, 8),
        fitness_func=fitness_func,
        fitness_limit=6,
        printer=print_stats
    )